# **Web Server Assigment**

# 1




## Overview
In this assignment, you will explore the fundamentals of socket programming for TCP connections in Python. You will learn how to create a socket, bind it to a specific address and port, and send and receive HTTP packets. Additionally, you will gain an understanding of the basic structure of HTTP header formats.

Your task is to develop a web server that processes one HTTP request at a time. The web server should:

1. Accept and parse the incoming HTTP request.
2. Retrieve the requested file from the server’s file system.
3. Construct an HTTP response message comprising the requested file preceded by appropriate header lines.
4. Send the response directly to the client.

If the requested file is not found on the server, the web server must return an HTTP "404 Not Found" message to the client.

## Code
Below is a skeleton code for the web server. Your task is to complete the code. The sections where you need to add your implementation are marked with `#-----#`. Some sections may require multiple lines of code.

## Running the Server
1. Place an HTML file (e.g., `HelloWorld.html`) in the same directory as the server program.
2. Execute the server program.
3. Determine the IP address of the machine running the server (e.g., `128.238.251.26`).
4. From a different machine or the same machine, open a web browser and navigate to the URL corresponding to the server. For example:
   ```
   http://128.238.251.26:6789/HelloWorld.html
   ```
   In this example, `HelloWorld.html` is the name of the HTML file you placed in the server directory. Note the port number `6789` used after the colon. Replace this port number with the one configured in your server code.

5. The browser should display the contents of `HelloWorld.html`. If the port number is omitted (e.g., `http://128.238.251.26/HelloWorld.html`), the browser will default to port 80. In this case, ensure your server is listening on port 80.
6. Test requesting a file that is not present on the server. You should receive a "404 Not Found" message in the browser.


## What to Submit
Submit the following items:

1. The complete server code.
2. Screenshots of your client browser demonstrating:
   - Successful retrieval of the HTML file content from the server.
   - Receiving a "404 Not Found" message for a non-existent file.

---

In [ ]:
#import socket module
import socket
from socket import *
import sys # In order to terminate the program

serverPort = 12000
serverSocket = socket(AF_INET, SOCK_STREAM)
serverSocket.bind(("", serverPort))
serverSocket.listen(1)
#Prepare a sever socket
#_________#
print ('Ready to serve...')
while True:
    #Establish the connection
    print ('Ready to serve...')
    connectionSocket, addr = serverSocket.accept()  
    try:

        message = connectionSocket.recv(1024).decode()  # Receive message from client
        print(message)
        filename = message.split()[1]  # Extract the filename from the request
        f = open(filename[1:])  # Open the requested file (skip the leading '/')
        outputdata = f.read()  # Read file contents
        f.close()  # Close the file after reading

        connectionSocket.send("HTTP/1.1 200 OK\r\n\r\n".encode())

        # Send the content of the requested file to the client
        connectionSocket.send(outputdata.encode())  # Send the entire file content at once
        connectionSocket.send("\r\n".encode())  # End the HTTP response

        connectionSocket.close()  # Close the client socket after sending data

    except IOError:
        # If file is not found, send 404 Not Found error message
        connectionSocket.send("HTTP/1.1 404 Not Found\r\n\r\n".encode())
        connectionSocket.send("<html><head></head><body><h1>404 Not Found</h1></body></html>".encode())

        connectionSocket.close() 
serverSocket.close()
sys.exit() #Terminate the program after sending the corresponding data

Ready to serve...
Ready to serve...


# 2

Currently, the web server handles only one HTTP request at a time. Implement a multithreaded server
that is capable of serving multiple requests simultaneously. Using threading, first create a main thread
in which your modified server listens for clients at a fixed port. When it receives a TCP connection
request from a client, it will set up the TCP connection through another port and services the client
request in a separate thread. There will be a separate TCP connection in a separate thread for each
request/response pair.

In [ ]:
import socket
from socket import *
import threading
import sys  # In order to terminate the program

serverPort = 12001
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.bind(("", serverPort))
serverSocket.listen(1)

print('Ready to serve...')


def handle_client(connectionSocket, addr):
    try:
        message = connectionSocket.recv(1024).decode() 
        print(f"Request from {addr}: {message}")
        filename = message.split()[1]  
        
        
        try:
            f = open(filename[1:])  
            outputdata = f.read() 
            f.close()  

            
            connectionSocket.send("HTTP/1.1 200 OK\r\n\r\n".encode())
            connectionSocket.send(outputdata.encode())  
            connectionSocket.send("\r\n".encode()) 

        except IOError:
           
            connectionSocket.send("HTTP/1.1 404 Not Found\r\n\r\n".encode())
            connectionSocket.send("<html><head></head><body><h1>404 Not Found</h1></body></html>".encode())
    
    except Exception as e:
        print(f"Error handling request from {addr}: {e}")
    
    finally:
        connectionSocket.close()  


while True:
    connectionSocket, addr = serverSocket.accept()  
   
   
    client_thread = threading.Thread(target=handle_client, args=(connectionSocket, addr))
    client_thread.start()


serverSocket.close()
sys.exit()  # Terminate the program after closing the server socket


Ready to serve...


# 3

The Multi Thread Web Server is now set and working but cannot handle changes in the source files. How can we look for changes in the source directory and access it in your code ? Explore Libraries in python

In [ ]:
import socket
import threading
import os
import sys
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time

serverPort = 12002
serverSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serverSocket.bind(("", serverPort))
serverSocket.listen(1)
print('Server ready to serve...')


file_cache = {}

class FileChangeHandler(FileSystemEventHandler):
    def on_modified(self, event):
        if event.src_path and not event.is_directory:
            filename = os.path.basename(event.src_path)
            print(f"File updated: {filename}")
            file_cache.pop(filename, None)  

def start_watching():
    observer = Observer()
    observer.schedule(FileChangeHandler(), ".", recursive=False)
    observer.start()
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()


threading.Thread(target=start_watching, daemon=True).start()

def handle_client(connectionSocket, addr):
    try:
        message = connectionSocket.recv(1024).decode()
        print(f"Request from {addr}: {message}")
        filename = message.split()[1].lstrip("/")
        
        if filename not in file_cache:
            if os.path.isfile(filename):
                with open(filename, "r") as f:
                    file_cache[filename] = f.read()
            else:
                connectionSocket.send("HTTP/1.1 404 Not Found\r\n\r\n".encode())
                connectionSocket.send("<html><body><h1>404 Not Found</h1></body></html>".encode())
                return

        connectionSocket.send("HTTP/1.1 200 OK\r\n\r\n".encode())
        connectionSocket.send(file_cache[filename].encode())
        connectionSocket.send("\r\n".encode())
    except Exception as e:
        print(f"Error handling request from {addr}: {e}")
    finally:
        connectionSocket.close()

try:
    while True:
        connectionSocket, addr = serverSocket.accept()
        print(f"Accepted connection from {addr}")
        threading.Thread(target=handle_client, args=(connectionSocket, addr)).start()
except KeyboardInterrupt:
    print("Shutting down the server...")
finally:
    serverSocket.close()
    sys.exit()


After you've accessed it in your code, Post a notification on the hosted http page to reload if any source directory changes

In [ ]:
import socket
import threading
import os
import sys
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time
from websocket_server import WebsocketServer  


serverPort = 12002
serverSocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serverSocket.bind(("", serverPort))
serverSocket.listen(1)
ws_clients = []
ws_server = WebsocketServer(host="", port=12003)  



print('Server ready to serve...')


file_cache = {}
clients = set() 
class FileChangeHandler(FileSystemEventHandler):
    def on_modified(self, event):
        global clients


        if event.src_path and not event.is_directory:
            filename = os.path.basename(event.src_path)
            print(f"File updated: {filename}")
            file_cache.pop(filename, None)  
            
            notify_clients("RELOAD")

            

 
    


def start_watching():
    observer = Observer()
    observer.schedule(FileChangeHandler(), ".", recursive=False)
    observer.start()
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()


threading.Thread(target=start_watching, daemon=True).start()
def new_client(client, server):
    """Handles new WebSocket connections."""
    ws_clients.append(client)
    print(f"WebSocket client connected: {client}")

def client_left(client, server):
    """Handles WebSocket client disconnection."""
    ws_clients.remove(client)
    print(f"WebSocket client disconnected: {client}")

def notify_clients(message):
    """Send a notification to all WebSocket clients."""
    for client in ws_clients:
        try:
            ws_server.send_message(client, message)
        except:
            ws_clients.remove(client)  

def start_ws_server():
    ws_server.set_fn_new_client(new_client)
    ws_server.set_fn_client_left(client_left)
    ws_server.run_forever()
threading.Thread(target=start_ws_server, daemon=True).start()



def handle_client(connectionSocket, addr):
    clients.add(connectionSocket)  
    try:
        message = connectionSocket.recv(1024).decode()
        print(f"Request from {addr}: {message}")
        filename = message.split()[1].lstrip("/")
        
        if filename not in file_cache:
            if os.path.isfile(filename):
                with open(filename, "r") as f:
                    file_cache[filename] = f.read()
            else:
                connectionSocket.send("HTTP/1.1 404 Not Found\r\n\r\n".encode())
                connectionSocket.send("<html><body><h1>404 Not Found</h1></body></html>".encode())
                return

        connectionSocket.send("HTTP/1.1 200 OK\r\n\r\n".encode())
        
        modified_content = file_cache[filename] + """
        <script>
        const ws = new WebSocket("ws://localhost:12003");
        ws.onmessage = function(event) {
        if (event.data === "RELOAD") {
         alert("A file has been updated. Please refresh the page manually.");
                                      }
                  };
        </script>
"""

        connectionSocket.send(modified_content.encode())

        connectionSocket.send("\r\n".encode())
    except Exception as e:
        print(f"Error handling request from {addr}: {e}")
    finally:
        clients.discard(connectionSocket)
        connectionSocket.close()

try:
    while True:
        connectionSocket, addr = serverSocket.accept()
        print(f"Accepted connection from {addr}")
        threading.Thread(target=handle_client, args=(connectionSocket, addr)).start()
except KeyboardInterrupt:
    print("Shutting down the server...")
finally:
    serverSocket.close()
    sys.exit()


Your dynamic http server is now ready